In [1]:
import os
import shutil

In [4]:
!rm -rf ../pre/count_words

In [5]:
data_list = os.listdir("../data")
data_list = list(data_list)

os.makedirs(f'../pre/', exist_ok=True)
os.makedirs(f'../pre/count_words', exist_ok=True)
for file in data_list:
    shutil.copyfile(f'../data/{file}/txt/{file}.txt', f'../pre/count_words/{file}.txt')

In [ ]:
!ls ../data/ | wc -w
!ls ../pre/count_words | wc -w

In [8]:
word_count = {}
file_list = os.listdir("../pre/count_words")

for file in file_list:
    with open(f'../pre/count_words/{file}', 'r') as f:
        text = f.read()
        for word in text.split():
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

In [ ]:
print(word_count['AI'])